### 參考自https://huggingface.co/docs/trl/v0.4.7/en/sft_trainer
#### 1. def formatting_prompts_func(example):

------------------
###

### 1. 安裝套件

In [1]:
!pip install transformers accelerate trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 7.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1

### 2. 讀取訓練集

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pickle

data_path = "/content/drive/MyDrive/1122自然語言處理/自然語言處理期末報告/Data"
data_file = "dataframe.pkl"
with open(data_path + "/" + data_file, 'rb') as f:
    df = pickle.load(f)

df

,abstract,y
0,A fully differential calculation in perturba...,A fully differential calculation in perturbati...
1,"We describe a new algorithm, the $(k,\ell)$-...","We describe a new algorithm, the $(k,\ell)$-pe..."
2,The evolution of Earth-Moon system is descri...,The evolution of Earth-Moon system is describe...
3,We show that a determinant of Stirling cycle...,We show that a determinant of Stirling cycle n...
4,In this paper we show how to compute the $\L...,In this paper we show how to compute the $\Lam...
...,...,...
95,In this note we present three representation...,In this note we present three representations ...
96,We review recent progress in operator algebr...,Our emphasis is on use of representation theor...
97,"Sparse Code Division Multiple Access (CDMA),...","Sparse Code Division Multiple Access (CDMA), a..."
98,For positive semidefinite matrices $A$ and $...,"In the course of this work, we introduce the n..."


### 3. 設定HF_TOKEN

In [ ]:

#https://www.pythontutorial.net/python-basics/python-read-text-file/

import os
import json

filepath = os.path.dirname(os.getcwd()) + "\\" + "Resource" #"C:/Users/UncleCarter/source/reposPython/CS_Fury_NLP"

filename = "hf_access_token.txt"
hf_access_token = ''
with open(filepath + "\\" + filename, encoding="utf8")as f:
     hf_access_token = f.read()



In [ ]:
from google.colab import userdata
hf_access_token = userdata.get('HF_TOKEN')

### 4. 讀取model and tokenizer

In [ ]:
import os
import logging
import time
import argparse
from typing import Optional, Union
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset


model_openELM_270M = "apple/OpenELM-270M"
model_openELM_270M_revision = "945fb18d02b1c4c81d7989e80b7564c6d91e8300"

model_openELM_450M_Instruct = "apple/OpenELM-450M-Instruct"
model_openELM_450M_Instruct_revision = "ec0350ea7200b718fc2168320a4826f17630c5b1"

model_openELM_1B = "apple/OpenELM-1_1B-Instruct"
model_openELM_1B_revision = "24982bbb6f27ae000566a6892cf669e0e06278ae"

model_openELM_3B = "apple/OpenELM-3B-Instruct"
model_openELM_3B_revision = "d3c76da586450c73898e6bed70cecf3376300fb3"

model_Llama_7B = "meta-llama/Llama-2-7b-hf"
model_Llama_7B_revision = "01c7f73d771dfac7d292323805ebc428287df4f9"

model_checkpoint = model_openELM_270M
model_revision = model_openELM_270M_revision
tokenizer_checkpoint = model_Llama_7B

device = "cuda" if torch.cuda.is_available() else "cpu"

#(1) build tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_checkpoint,
    token=hf_access_token,
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
        model_checkpoint,
        revision=model_revision,
        trust_remote_code=True
)

# model.to(device)

model.save_pretrained('./default_model')
tokenizer.save_pretrained('./default_model')


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

configuration_openelm.py:   0%|          | 0.00/14.3k [00:00<?, ?B/s]

modeling_openelm.py:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

('./default_model/tokenizer_config.json',
 './default_model/special_tokens_map.json',
 './default_model/tokenizer.model',
 './default_model/added_tokens.json',
 './default_model/tokenizer.json')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("./default_model/", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("./default_model/", trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/apple/OpenELM-270M:
- configuration_openelm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


### 5. Trainer

In [ ]:
from transformers import set_seed, get_constant_schedule
from trl import SFTTrainer
from datasets import Dataset

tokenizer.add_special_tokens({'pad_token': '[PAD]'})
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['abstract'])):
        text = f"### Question: {example['abstract'][i]}\n ### Answer: {example['y'][i]}"
        output_texts.append(text)
    return output_texts


training_arguments = TrainingArguments(
    output_dir='./results',
    push_to_hub=False,
    report_to = "none",
    seed = 42,
)

trainer = SFTTrainer(
    model,
    train_dataset=Dataset.from_pandas(df),
    formatting_func=formatting_prompts_func,
    #dataset_text_field="text",
    args=training_arguments,
    max_seq_length=512,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

#### 5.1. Save out Trained model and tokenizer

In [ ]:
trainer.save_model('./trained_model')
#=========================================
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')

### 6. Implement your main function here

In [ ]:
import time
import torch



model_openELM_270M = "apple/OpenELM-270M"
model_openELM_270M_revision = "945fb18d02b1c4c81d7989e80b7564c6d91e8300"

model_openELM_1B = "apple/OpenELM-1_1B-Instruct"
model_openELM_1B_revision = "24982bbb6f27ae000566a6892cf669e0e06278ae"

model_openELM_3B = "apple/OpenELM-3B-Instruct"
model_openELM_3B_revision = "d3c76da586450c73898e6bed70cecf3376300fb3"

model_Llama_7B = "meta-llama/Llama-2-7b-hf"
model_Llama_7B_revision = "01c7f73d771dfac7d292323805ebc428287df4f9"

def generate(prompt=None, model=None, tokenizer=None):

    # model_checkpoint = modelChosen
    # model_revision = modelRevision
    # tokenizer_checkpoint = tokenizerChosen

    device = "cuda" if torch.cuda.is_available() else "cpu"
    #(1) build tokenizer
    # tokenizer = AutoTokenizer.from_pretrained(
    #     tokenizer_checkpoint,
    #     token=hf_access_token
    # )
    # model = AutoModelForCausalLM.from_pretrained(
    #         model_checkpoint,
    #         revision=model_revision,
    #         trust_remote_code=True
    # )
    # model.to(device).eval()

    #(2) Prepare the prompt and form tokenized_prompt
    tokenized_prompt = tokenizer(prompt) #truncation=True, padding=True

    tokenized_prompt = torch.tensor(
        tokenized_prompt['input_ids'],
        device=device
    )
    tokenized_prompt = tokenized_prompt.unsqueeze(0)


    #(3) build model
    stime = time.time()
    output_ids = model.generate(
        tokenized_prompt,
        max_length=1024,
        pad_token_id=0,
        assistant_model=None
    )
    output_text = tokenizer.decode(
        output_ids[0][tokenized_prompt.shape[1]:].tolist(),
        skip_special_tokens=True
    )
    generation_time = time.time() - stime

    return output_text, generation_time


In [ ]:

def extract_sentence(abstract: str) -> str:
    prompt = "From the following abstract, extract the sentences that shows the methods of the research. Only the sentences from the abstract, no other information.\n\n\n```%s```" % abstract
    output_text, genertaion_time = generate(
        prompt=prompt,
        tokenizer=tokenizer,
        model=model,
        #hf_access_token=hf_access_token #userdata.get('HF_TOKEN')
    )
    return output_text

In [ ]:
abstract = """The reliability of self-labeled data is an important issue when the data are regarded as ground-truth for training and testing learning-based models.
This paper addresses the issue of false-alarm hashtags in the self-labeled data for irony detection.
We analyze the ambiguity of hashtag usages and propose a novel neural network-based model, which incorporates linguistic information from different aspects, to disambiguate the usage of three hashtags that are widely used to collect the training data for irony detection.
Furthermore, we apply our model to prune the self-labeled training data.
Experimental results show that the irony detection model trained on the less but cleaner training instances outperforms the models trained on all data."""

predicted = extract_sentence(abstract)
print(predicted)

### 7. Evaluation
#### apple/OpenELM-270M，訓練過後有16%的結果

In [ ]:
#!pip install rouge-score

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rougeL'])

In [ ]:
reference = """We analyze the ambiguity of hashtag usages and propose a novel neural network-based model, which incorporates linguistic information from different aspects, to disambiguate the usage of three hashtags that are widely used to collect the training data for irony detection. Furthermore, we apply our model to prune the self-labeled training data."""

print(scorer.score(reference, predicted)['rougeL'].fmeasure)

0.16987179487179488


In [ ]:
predicted

'\n\n\n```The reliability of self-labeled data is an important issue when the data are regarded as ground-truth for training and testing learning-based models.\nThis paper addresses the issue of false-alarm hashtags in the self-labeled data for irony detection.\nWe analyze the ambiguity of hashtag usages and propose a novel neural network-based model, which incorporates linguistic information from different aspects, to disambiguate the usage of three hashtags that are widely used to collect the training data for irony detection.\nFurthermore, we apply our model to prune the self-labeled training data.\nExperimental results show that the irony detection model trained on the less but cleaner training instances outperforms the models trained on all data.```\n\n\n```The reliability of self-labeled data is an important issue when the data are regarded as ground-truth for training and testing learning-based models.\nThis paper addresses the issue of false-alarm hashtags in the self-labeled d